In [1]:
import geopandas as gpd
import xarray as xr
import pandas as pd

import pint_xarray
import pint_pandas
import pint

from summaflow import (
    GeoLayer,
    SUMMAWorkflow,
)

import os
import glob

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

# forcings path
root_path_forcings = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings'

In [3]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

In [4]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

# dummy layers for `tan_slope`, `contourLength` and `downHRUindex`
# until relevant workflows are implemented inside `gistool`
slope = elv
contour = elv
hru_index = elv

In [5]:
exp = SUMMAWorkflow(
    forcing_data = glob.glob(os.path.join(root_path_forcings, '**', '*.nc'), recursive=True),
    forcing_name_mapping = {
        'pptrate': 'precipitation',
        'airtemp': 'air_temperature',
    },
    forcing_unit_mapping = {
        'precipitation': 'meters/hour',
        'air_temperature': 'degC',
    },
    forcing_to_unit_mapping = {
        'precipitation': 'mm/s',
        'air_temperature': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meters',
        'forcing_time_zone': 'utc', # original timezone of the forcing datatset
        'target_time_zone': 'America/Edmonton', # if UTC, SUMMA converts to local time zone internally
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface blah blah',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data = {
        'elevation': elv,
        'soilTypeIndex': soil,
        'vegTypeIndex': landcover,
        'tan_slope': slope,
        'contourLength': contour,
        'downHRUindex': hru_index,
    },
    settings = {
        'model_path': '/Users/kasrakeshavarz/Desktop/test',
    },
    cold_state = {
        'layers': {
            'nSoil': 8,
            'nSnow': 0,
        },
        'states': {
            'scalarCanopyIce': 0,
            'scalarCanopyLiq': 0,
            'scalarSnowDepth': 0,
            'scalarSWE': 0,
            'scalarSfcMeltPond': 0,
            'scalarAquiferStorage': 0.4,
            'scalarSnowAlbedo': 0,
            'scalarCanairTemp': 283.16,
            'scalarCanopyTemp': 283.16,
            'mLayerTemp': 283.16,
            'mLayerVolFracIce': 0,
            'mLayerVolFracLiq': 0.4,
            'mLayerMatricHead': -1.0,
            'mLayerDepth': [0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1, 1.5],
        },
    }
)

In [6]:
exp

Forcing files: ['/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201101.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201102.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201103.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201104.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201105.nc', '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/forcings/Bow_at_Banff_ERA5_remapped_domain_Bow_at_Banff_ERA5_merged_201106.nc', '/User

## Workflow tests

In [7]:
exp.init_attrs(return_ds=False)

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:127: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


In [8]:
exp.init_forcing(return_ds=False)

In [9]:
exp.init_cold_state(return_ds=False)

In [14]:
exp.cold_state

<xarray.Dataset> Size: 7kB
Dimensions:  (hru: 169, scalarv: 1, midSoil: 8, midToto: 8, ifcToto: 9)
Coordinates:
  * hru      (hru) int64 1kB 71027942 71027957 71027962 ... 71039250 71039266
  * midSoil  (midSoil) int64 64B 0 1 2 3 4 5 6 7
  * midToto  (midToto) int64 64B 0 1 2 3 4 5 6 7
  * ifcToto  (ifcToto) int64 72B 0 1 2 3 4 5 6 7 8
  * scalarv  (scalarv) int64 8B 0
Data variables:
    hruId    (hru) int64 1kB 71027942 71027957 71027962 ... 71039250 71039266
    dt_init  (scalarv, hru) int64 1kB 3600 3600 3600 3600 ... 3600 3600 3600
    nSoil    (scalarv, hru) int64 1kB 8 8 8 8 8 8 8 8 8 8 ... 8 8 8 8 8 8 8 8 8
    nSnow    (scalarv, hru) int64 1kB 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
Attributes: (12/19)
    hruId:                 {'long_name': 'Index of hydrological response unit...
    dt_init:               {'long_name': 'Time step size of forcing data', 'u...
    nSoil:                 {'long_name': 'Number of soil layers', 'units': 'd...
    nSnow:                 {'long_name': 'Number of snow layers', 'units': 'd...
    scalarCanopyIce:       {'long_name': 'Mass of ice on the vegetation canop...
    scalarCanopyliq:       {'long_name': 'Mass of liquid water on the vegetat...
    ...                    ...
    mLayerDepth:           {'long_name': 'Depth of each layer', 'units': 'met...
    iLayerHeight:          {'long_name': 'Height of the layer interface; top ...
    mLayerTemp:            {'long_name': 'Temperature of each layer', 'units'...
    mLayerVolFracIce:      {'long_name': 'Volumetric fraction of ice in each ...
    mLayerVolFracLiq:      {'long_name': 'Volumetric fraction of liquid water...
    mLayerMatricHead:      {'long_name': 'Matric head of water in the soil', ...

In [16]:
import pandas as pd

# Create date range
date_range = pd.date_range(start='1980-01-01', end='1981-12-31 23:59:59', freq='3h')

# Create DataFrame
df = pd.DataFrame({'datetime': date_range})

print(df.head())
print("\n...\n")
print(df.tail())
print(f"\nTotal rows: {len(df)}")

             datetime
0 1980-01-01 00:00:00
1 1980-01-01 03:00:00
2 1980-01-01 06:00:00
3 1980-01-01 09:00:00
4 1980-01-01 12:00:00

...

                datetime
5843 1981-12-31 09:00:00
5844 1981-12-31 12:00:00
5845 1981-12-31 15:00:00
5846 1981-12-31 18:00:00
5847 1981-12-31 21:00:00

Total rows: 5848


In [18]:
pd.infer_freq(date_range)

'3h'

In [22]:
int(pd.to_timedelta('3h').total_seconds())

10800